In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
ccf_df = pd.read_csv("../data/credit_card_fraud.csv")

In [3]:
ccf_df = pd.concat([ccf_df]*100).reset_index(drop=True)

In [4]:
ccf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 20 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Transaction Date and Time                800000 non-null  object 
 1   Transaction Amount                       800000 non-null  float64
 2   Cardholder Name                          800000 non-null  object 
 3   Card Number (Hashed or Encrypted)        800000 non-null  object 
 4   Merchant Name                            800000 non-null  object 
 5   Merchant Category Code (MCC)             800000 non-null  int64  
 6   Transaction Location (City or ZIP Code)  800000 non-null  object 
 7   Transaction Currency                     800000 non-null  object 
 8   Card Type                                800000 non-null  object 
 9   Card Expiration Date                     800000 non-null  object 
 10  CVV Code (Hashed or Encrypted)  

In [5]:
ccf_df["Card Type"].unique()

array(['MasterCard', 'American Express', 'Visa'], dtype=object)

In [6]:
ccf_df["Transaction Currency"].unique()

array(['INR', 'EUR', 'USD'], dtype=object)

In [7]:
ccf_df.head()

,Transaction Date and Time,Transaction Amount,Cardholder Name,Card Number (Hashed or Encrypted),Merchant Name,Merchant Category Code (MCC),Transaction Location (City or ZIP Code),Transaction Currency,Card Type,Card Expiration Date,CVV Code (Hashed or Encrypted),Transaction Response Code,Transaction ID,Fraud Flag or Label,Previous Transactions,Transaction Source,IP Address,Device Information,User Account Information,Transaction Notes
0,2022-09-24 13:54:27,285.88,Shray Soman,daca51bffe0fc4eaaa7c430917f94b75fd893712492254...,"Rajagopalan, Ghose and Kant",3590,Khammam,INR,MasterCard,04/29,01299ac65733b5a3d774265fbfe8396b8611e5e3321855...,5,b7f69cbc-a03d-41f8-adca-75920b0242c3,1,NaN,Online,18.106.240.6,Tablet,NaN,Consequatur corporis minima ad vero deserunt n...
1,2020-07-24 11:20:13,1777.32,Lakshit Bakshi,d5366dd9be3a0266c0252baceaa6332210ed4dbd0ef252...,Sule PLC,7277,Vasai-Virar,EUR,American Express,03/26,ad8fa913d25b3970c6efa8ca504da8ba670ce2a9cf012d...,12,ad53cc8e-8412-422e-8cad-4176daac8387,1,NaN,Online,212.48.185.128,Mobile,jloyal,Dolore repellendus odio deleniti. Eaque quibus...
2,2023-03-18 01:05:36,3939.01,Riya Bobal,4737a6384aa1c3c31f7768b86633d6a8401dd6ab4a8ff1...,Badal PLC,9297,Nangloi Jat,USD,Visa,11/29,eb3be230bbd2844b1f5d8f2e4fab9ffba8ab22cfeeb69c...,5,f09cecd6-0d74-4551-8089-a31d64e6b9c5,1,3 or more,In-Person,17.190.112.46,Mobile,NaN,Unde beatae perspiciatis sapiente. Voluptates ...
3,2021-01-07 21:53:04,376.44,Mohanlal Balakrishnan,4721806eed8f2663bb597ff13e79a294de318358fbb54b...,Konda-Sodhi,5686,Ramagundam,USD,Visa,09/25,891d46993a36d78392247c642138cede01d9841daab1d9...,5,b208ae0b-4c4f-428f-b6b9-5360b288b947,0,2,In-Person,153.136.24.104,Tablet,NaN,Cupiditate repellendus necessitatibus quo occa...
4,2021-12-16 06:22:24,1687.33,Mannat Rout,9deacc3a9efd6e382826d400620aca5f23ed94327578e5...,Dua Ltd,2940,Adoni,INR,MasterCard,03/31,48a1a756f2d83f1dc57bbf14052b70a6f40d0fceed6662...,5,6b4e4e43-5b73-4906-9973-299a1b2a5e71,1,2,Online,196.153.28.131,Desktop,NaN,Molestias assumenda consectetur itaque veritat...


# Caso de estudio
Nos piden sacar un informe con la cantidad de cada transacción quitandole las comisiones correspondientes a la tarjeta de crédito, las correspondientes a cambio de moneda y finalmente queremos expresar el resultado en Euros. A esta cantidad la llamaremos transacción neta. Para ello nos proporcionan los siguientes datos:

- Comisiones según la tarjeta:
    - MasterCard = 2%
    - Visa = 3 %
    - American Express = 5%
- Comisión por pago en moneda extranjera:
    - INR = 1%
    - USD = 0.5 %
- Tasa conversión a Euros:
    - INR/EUR = 0.011
    - USD/EUR = 0.95

## Resolución iterando el dataframe

In [8]:
for index, row in ccf_df.iterrows():
    tarjeta = row["Card Type"]
    moneda = row["Transaction Currency"]
    if tarjeta == "MasterCard":
        comision_tarjeta = 0.02
    elif tarjeta == "Visa":
        comision_tarjeta = 0.03
    elif tarjeta == "American Express":
        comision_tarjeta = 0.05
    if moneda == "INR":
        comision_moneda = 0.01
        conversion_moneda = 0.011
    elif moneda == "USD":
        comision_moneda = 0.005
        conversion_moneda = 0.95
    elif moneda == "EUR":
        comision_moneda = 0
        conversion_moneda = 1
    ccf_df.loc[index, "transaccion_neta"] = row["Transaction Amount"] * (1-comision_tarjeta-comision_moneda) * conversion_moneda

In [9]:
ccf_df["transaccion_neta"].head()

0       3.050340
1    1688.454000
2    3611.087417
3     345.101370
4      18.003811
Name: transaccion_neta, dtype: float64

In [10]:
ccf_df = ccf_df.drop(columns={"transaccion_neta"})

## Usando Apply

In [11]:
def calcular_transaccion_neta(transaction_amount, tarjeta, moneda):
    if tarjeta == "MasterCard":
        comision_tarjeta = 0.02
    elif tarjeta == "Visa":
        comision_tarjeta = 0.03
    elif tarjeta == "American Express":
        comision_tarjeta = 0.05
    if moneda == "INR":
        comision_moneda = 0.01
        conversion_moneda = 0.011
    elif moneda == "USD":
        comision_moneda = 0.005
        conversion_moneda = 0.95
    elif moneda == "EUR":
        comision_moneda = 0
        conversion_moneda = 1
    return transaction_amount * (1-comision_tarjeta-comision_moneda) * conversion_moneda

In [12]:
ccf_df.head()

,Transaction Date and Time,Transaction Amount,Cardholder Name,Card Number (Hashed or Encrypted),Merchant Name,Merchant Category Code (MCC),Transaction Location (City or ZIP Code),Transaction Currency,Card Type,Card Expiration Date,CVV Code (Hashed or Encrypted),Transaction Response Code,Transaction ID,Fraud Flag or Label,Previous Transactions,Transaction Source,IP Address,Device Information,User Account Information,Transaction Notes
0,2022-09-24 13:54:27,285.88,Shray Soman,daca51bffe0fc4eaaa7c430917f94b75fd893712492254...,"Rajagopalan, Ghose and Kant",3590,Khammam,INR,MasterCard,04/29,01299ac65733b5a3d774265fbfe8396b8611e5e3321855...,5,b7f69cbc-a03d-41f8-adca-75920b0242c3,1,NaN,Online,18.106.240.6,Tablet,NaN,Consequatur corporis minima ad vero deserunt n...
1,2020-07-24 11:20:13,1777.32,Lakshit Bakshi,d5366dd9be3a0266c0252baceaa6332210ed4dbd0ef252...,Sule PLC,7277,Vasai-Virar,EUR,American Express,03/26,ad8fa913d25b3970c6efa8ca504da8ba670ce2a9cf012d...,12,ad53cc8e-8412-422e-8cad-4176daac8387,1,NaN,Online,212.48.185.128,Mobile,jloyal,Dolore repellendus odio deleniti. Eaque quibus...
2,2023-03-18 01:05:36,3939.01,Riya Bobal,4737a6384aa1c3c31f7768b86633d6a8401dd6ab4a8ff1...,Badal PLC,9297,Nangloi Jat,USD,Visa,11/29,eb3be230bbd2844b1f5d8f2e4fab9ffba8ab22cfeeb69c...,5,f09cecd6-0d74-4551-8089-a31d64e6b9c5,1,3 or more,In-Person,17.190.112.46,Mobile,NaN,Unde beatae perspiciatis sapiente. Voluptates ...
3,2021-01-07 21:53:04,376.44,Mohanlal Balakrishnan,4721806eed8f2663bb597ff13e79a294de318358fbb54b...,Konda-Sodhi,5686,Ramagundam,USD,Visa,09/25,891d46993a36d78392247c642138cede01d9841daab1d9...,5,b208ae0b-4c4f-428f-b6b9-5360b288b947,0,2,In-Person,153.136.24.104,Tablet,NaN,Cupiditate repellendus necessitatibus quo occa...
4,2021-12-16 06:22:24,1687.33,Mannat Rout,9deacc3a9efd6e382826d400620aca5f23ed94327578e5...,Dua Ltd,2940,Adoni,INR,MasterCard,03/31,48a1a756f2d83f1dc57bbf14052b70a6f40d0fceed6662...,5,6b4e4e43-5b73-4906-9973-299a1b2a5e71,1,2,Online,196.153.28.131,Desktop,NaN,Molestias assumenda consectetur itaque veritat...


In [13]:
%%timeit
ccf_df["transaccion_neta"] = ccf_df.apply(lambda row: calcular_transaccion_neta(row["Transaction Amount"], row["Card Type"], row["Transaction Currency"]), axis=1)

2.49 s ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
ccf_df["transaccion_neta"].head()

0       3.050340
1    1688.454000
2    3611.087417
3     345.101370
4      18.003811
Name: transaccion_neta, dtype: float64

In [15]:
ccf_df = ccf_df.drop(columns={"transaccion_neta"})

## Operar vectorialmente

### Generar columnas con los valores de comisión y conversión (MAP IT!)

In [16]:
comisiones_tarjeta = {"MasterCard": 0.02, "Visa": 0.03, "American Express": 0.05}
comisiones_moneda = {"INR": 0.01, "USD": 0.005, "EUR": 0}
conversiones_moneda = {"INR": 0.011, "USD": 0.95, "EUR": 1}

In [17]:
ccf_df["comisiones_tarjeta"] = ccf_df["Card Type"].map(comisiones_tarjeta)
ccf_df["comisiones_moneda"] = ccf_df["Transaction Currency"].map(comisiones_moneda)
ccf_df["conversiones_moneda"] = ccf_df["Transaction Currency"].map(conversiones_moneda)

In [18]:
ccf_df.head()

,Transaction Date and Time,Transaction Amount,Cardholder Name,Card Number (Hashed or Encrypted),Merchant Name,Merchant Category Code (MCC),Transaction Location (City or ZIP Code),Transaction Currency,Card Type,Card Expiration Date,CVV Code (Hashed or Encrypted),Transaction Response Code,Transaction ID,Fraud Flag or Label,Previous Transactions,Transaction Source,IP Address,Device Information,User Account Information,Transaction Notes,comisiones_tarjeta,comisiones_moneda,conversiones_moneda
0,2022-09-24 13:54:27,285.88,Shray Soman,daca51bffe0fc4eaaa7c430917f94b75fd893712492254...,"Rajagopalan, Ghose and Kant",3590,Khammam,INR,MasterCard,04/29,01299ac65733b5a3d774265fbfe8396b8611e5e3321855...,5,b7f69cbc-a03d-41f8-adca-75920b0242c3,1,NaN,Online,18.106.240.6,Tablet,NaN,Consequatur corporis minima ad vero deserunt n...,0.02,0.010,0.011
1,2020-07-24 11:20:13,1777.32,Lakshit Bakshi,d5366dd9be3a0266c0252baceaa6332210ed4dbd0ef252...,Sule PLC,7277,Vasai-Virar,EUR,American Express,03/26,ad8fa913d25b3970c6efa8ca504da8ba670ce2a9cf012d...,12,ad53cc8e-8412-422e-8cad-4176daac8387,1,NaN,Online,212.48.185.128,Mobile,jloyal,Dolore repellendus odio deleniti. Eaque quibus...,0.05,0.000,1.000
2,2023-03-18 01:05:36,3939.01,Riya Bobal,4737a6384aa1c3c31f7768b86633d6a8401dd6ab4a8ff1...,Badal PLC,9297,Nangloi Jat,USD,Visa,11/29,eb3be230bbd2844b1f5d8f2e4fab9ffba8ab22cfeeb69c...,5,f09cecd6-0d74-4551-8089-a31d64e6b9c5,1,3 or more,In-Person,17.190.112.46,Mobile,NaN,Unde beatae perspiciatis sapiente. Voluptates ...,0.03,0.005,0.950
3,2021-01-07 21:53:04,376.44,Mohanlal Balakrishnan,4721806eed8f2663bb597ff13e79a294de318358fbb54b...,Konda-Sodhi,5686,Ramagundam,USD,Visa,09/25,891d46993a36d78392247c642138cede01d9841daab1d9...,5,b208ae0b-4c4f-428f-b6b9-5360b288b947,0,2,In-Person,153.136.24.104,Tablet,NaN,Cupiditate repellendus necessitatibus quo occa...,0.03,0.005,0.950
4,2021-12-16 06:22:24,1687.33,Mannat Rout,9deacc3a9efd6e382826d400620aca5f23ed94327578e5...,Dua Ltd,2940,Adoni,INR,MasterCard,03/31,48a1a756f2d83f1dc57bbf14052b70a6f40d0fceed6662...,5,6b4e4e43-5b73-4906-9973-299a1b2a5e71,1,2,Online,196.153.28.131,Desktop,NaN,Molestias assumenda consectetur itaque veritat...,0.02,0.010,0.011


### Operadores matemáticos directamente

In [19]:
%%timeit
ccf_df["transaccion_neta"] = ccf_df["Transaction Amount"] * (1 - ccf_df["comisiones_tarjeta"] - ccf_df["comisiones_moneda"]) * ccf_df["conversiones_moneda"]

2.56 ms ± 26.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
ccf_df["transaccion_neta"].head()

0       3.050340
1    1688.454000
2    3611.087417
3     345.101370
4      18.003811
Name: transaccion_neta, dtype: float64

In [21]:
ccf_df = ccf_df.drop(columns={"transaccion_neta"})

### Todavía más rápido directamente con Numpy

In [22]:
%%timeit
ccf_df["transaccion_neta"] = ccf_df["Transaction Amount"].to_numpy() * (1 - ccf_df["comisiones_tarjeta"].to_numpy() - ccf_df["comisiones_moneda"].to_numpy()) * ccf_df["conversiones_moneda"].to_numpy()

1.41 ms ± 22.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
ccf_df = ccf_df.drop(columns={"transaccion_neta"})

## Cuidado al usar subsets de un dataframe

In [24]:
%%timeit
ccf_df.loc[ccf_df["Card Type"] == "Visa", "comisiones_tarjeta"] = ccf_df.loc[ccf_df["Card Type"] == "Visa", "comisiones_tarjeta"] + 0.01

49.2 ms ± 158 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%timeit
ccf_df["comisiones_tarjeta"] = ccf_df["comisiones_tarjeta"] + ((ccf_df["Card Type"] == "Visa") * 0.01)

23.3 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%%timeit
visa_add_map = {"MasterCard": 0, "Visa": 0.01, "American Express": 0}
ccf_df["comisiones_tarjeta"] = ccf_df["comisiones_tarjeta"] + ccf_df["Card Type"].map(comisiones_tarjeta)

22 ms ± 97 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Operaciones ya existentes en pandas o numpy

In [27]:
%%timeit
# min max normalizer
(ccf_df["Transaction Amount"] - ccf_df["Transaction Amount"].min()) / (ccf_df["Transaction Amount"].max() - ccf_df["Transaction Amount"].min())

1.96 ms ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [28]:
%%timeit
ccf_df["Transaction Amount"].mean()

383 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
%%timeit
ccf_df["Transaction Amount"].to_numpy().mean()

144 µs ± 3.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Strings

## Cuidado con las funciones de strings!

In [30]:
ccf_df["Card Expiration Date"].head()

0    04/29
1    03/26
2    11/29
3    09/25
4    03/31
Name: Card Expiration Date, dtype: object

In [31]:
ccf_df["Card Expiration Date"][0].split("/")

['04', '29']

In [32]:
%%timeit
ccf_df["Card Expiration Date"].apply(lambda x: x.split("/"))

98.1 ms ± 714 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
ccf_df["Card Expiration Date"].str.split("/")

118 ms ± 448 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%%timeit
[value.split('/') for value in ccf_df["Card Expiration Date"].tolist()]

75.8 ms ± 504 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
